In [1]:
import sys
import os
import json
sys.path.insert(0,f'{os.getcwd()}/../art_snob_primrose/')
from src.datastore_reader import DataStoreReader
from utilities.datastore_helpers import DataStoreInterface

dsi = DataStoreInterface(project='artsnob-1')

/Users/mike/Projects/envs/.env_deco_primrose/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [6]:
article = {
    'title': "Architecture behind scalable, dynamic art recommendations pt 1",
    'text': [
            {"body": """Since 2018, I’ve been working on a side project to explore better wall art recommendations. The existing sites had too much unsorted content, and since I work in ML in my day job, I wanted to build something better. Art Snob is the result of that work, a scalable search engine for art."""},
            {"body": """Art Snob starts with a call to action where you answer visual questions that establish your taste, then as you browse, your recommendations are progressively refined. I wanted to make some posts that talked through the ML architecture and hopefully have some useful discussion. First we'll talk about the general design and models, and in a later article we'll dive into search. At a high level the data architecture looks like this: """},
            {"image": """https://storage.googleapis.com/artsnob-image-scrape/blog/Deco%20Reco%20Architecture%20(1).png"""},
            {"header": """Scraping Service:"""},
            {"body": """I needed lots of art data to make this useful, so I found art sites with affiliate programs, and set up (ethical!) scrapers to scalably consume their site-mapped content. I built scrapy spider crawlers to look through the content weekly, and find anything that hadn’t been indexed in the art metadata storage. The scrapers collected the artwork images as well as any metadata, particularly search tags. The images were uploaded to Google cloud storage (behind a CDN layer), where cloud functions then resize them to optimize bandwidth when serving. """},
            {"header": """Data Persistence:"""},
            {"body": """Here I used Google’s datastore document storage database. It’s a noSQL option which is basically a wrapper around BigTable with some convenience functions. If I was starting this now, I would swap this layer for either Mongo or even mysql, but I wanted to explore using datastore since the cloud hosting seemed so appealing. The db is very fast, and scales well, but I would have preferred some indexing and querying flexibility, like what you get with Mongo. Much of this logic has to move into the application layer, which is a bit of a drag. """},
            {"header": """Recommendation Data Prep Batch Job:"""},
            {"body": """Here’s where the bulk of the machine learning occurs. Once I had a critical mass of art (100k works or so), I started building models to categorize, label, and index the art to make meaningful content recommendations. The image embeddings were built with a simple model, inception resnet v2, using tf hub. They were dimensionally reduced with a PCA step. The tag embeddings were made by starting with a tf-idf embedding score (treating each artwork as a document), then adding in an exponential “art popularity” term, which overweighted popular tags to each tag’s tf-idf score. The tag embeddings were also dimensionally reduced with a PCA step. The tag score model and the two PCA transformers were trained once on a bulk majority of data, and serialized into GCS for incremental usage without needing all the data in the pipeline."""},
            {"body": """The image and tag embeddings were concatenated and inserted into an annoy index for nearest neighbor search. The index is saved in GCS for future live look-ups, but the neighbors are also cached in datastore after each pipeline run for better recommendation performance.  """},
            {"body": """Finally a mini-batch k-means algorithm is used to classify art into categorical indicators. The features are simply the concatenated embedding vectors used in the indexing step. This model was initially built in an offline batch process with 100k works. To find ideal hyper parameters, I did a grid search where each hyperparameter set output into a small web app showing 10 images at each cluster center, along with the highest scoring tag names for the cluster center artwork. This allowed me to score, and write a human-readable label (based on the tags I was seeing) for each cluster. In the business they call this “human in the loop” machine learning, but let’s be honest, it’s tedious. There are always subjective calls being made, so the site includes the ability for users to give feedback on the algorithms categorical classification. This is gathered in the training pipeline, so future iterations can work to move artwork into classifications such that the overlap between “mislabeled” neighbors is minimized as part of the hyperparameter search. Once the hyperparameters were chosen, the same model was used for incremental updates. """},
            {"body": """All of the above jobs were written in a python framework for repeatable ML ops called Primrose ({1}). The framework allows you to build re-usable nodes for operations, and extricates the configuration such that much of the work becomes writing configuration files rather than writing long pythons scripts for everything you’d like to do. It makes re-training or incremental updates very simple. You can basically build one single image that references a configuration file to run any given job. There are lots of frameworks to check out if you’re interested in this kind of thing: prefect ({2}), dagster ({3}), and kubeflow ({4}) have similar ideas with differing levels of dev-ops required. Either way this sort of modular approach keeps things clean and simple."""},
            {"header": """Recommendation Service:"""},
            {"body": """As you can see in the purple “Recommendation Service” box, the client-facing API takes a search engine approach to recommendations, an indexing step followed by a ranking model. In this way the system is flexible toward using any model for either step. For indexing, I started with a very simple approach, the indexing operation finds the clusters that the user has been interested in via browsing history or explicit indicators, and up-front excludes clusters that they have disliked. Datastore has a reverse index built up for each cluster so you can pull down all the metadata keys for each cluster in a few milliseconds. """},
            {"body": """After the indexing call, the backend has a few thousand options in memory, so a ranking model is used to re-rank the options by interest before being returned to the user. The ranking uses a linear model, with weights that are tunable via AB tests on the backend. There are only 3 features for each item, number of exposures (for this user), number of liked neighbors, and number of dislikes neighbors. The number of liked/disliked neighbors features are 1 if each image is within the set of 20 nearest neighbors to each user’s liked or disliked works. So to recap: the number of exposures has a negative coefficient (the more you see something and don’t interact, the less you’ll be shown that item), a binary variable for whether it’s close to something you’ve liked before (a positive coefficient), and a binary variable for whether it’s close to something you’re disliked before (a negative coefficient). All the art is then ranked and returned to the client. There is also an initial random ranking which seeds any tie-breaks, so the user won’t see the same recommended feed over and over when they don’t interact with any content. """},
            {"body": """The above implementation is primed to be swapped out with a learning to rank model after member behavior starts informing the ideal content. Without many users this is very much a cold-start problem."""},
            {"body": """Give Art Snob a try for yourself, to see how your recommendations look! Stay tuned for updates on how the search uses the embeddings paired with text embeddings."""}
    ],
    "tags": ['software engineering', 'machine learning', 'architecture', 'art snob'],
    "author": {"name": "Mike Skarlinski, PhD", "title": "Art Snob Founder", "bio": "Mike has been building data products and leading data science teams over the last 9 years."},
    "publish_date":'Feb. 24, 2021',
    'references': ['https://github.com/ww-tech/primrose', 'https://www.prefect.io/', 'https://github.com/dagster-io/dagster', 'https://www.kubeflow.org/']
}

In [32]:
article = {
'title': "Data Science isn't a job",
'text': [
        {"body": """At Art Snob, recommending the right art is all about effectively utilizing data. It follows that good data science is critical, and I’ve been thinking a lot about what makes a data team successful. When reflecting back on teams I’ve managed and participated in, there are some common themes which have helped me grow in my own career, which I wanted to share here:"""},
        {'header': """“Data science” isn’t a job, it’s an outcome."""},
        {"body": """Over the last 10 years, almost every major corporation has decided they need to hire data scientists. This hiring zeal, however, has not been accompanied by widespread understanding of what data scientists actually do. In several organizations I’ve observed, management accepts this lack of understanding, preferring to view data scientists as masters of a kind of mystic art. “Not sure what goes on in that team,” the leadership says, “but sometimes we get cool stuff from them.”"""},
        {"body": """Organizational confusion is fueled by the fact that definitions of the term “data scientist” often cast a wide net across different skills, disciplines, and technologies. Academic and industry articles ({1},{2},{3},{4},{5}) that define data science often resort to defining data science as a long list of skills or functions as opposed to a cohesive practice. The nebulousness of our current definition of data science practitioners is illustrated by a popular tweet by Josh Williams:"""},
        {"subtext": """Data Scientist (n.): Person who is better at statistics than any software engineer and better at software engineering than any statistician."""},
        {"body": """In my experience, the confusion lies in mistakenly believing “data science” is a job. It isn’t. It’s an outcome: creating value from a company’s data. Organizations cannot extract value from their data if they look for candidates that possess an endless hodgepodge of skills,  certifications and education. These candidates don’t exist. Instead, organizations need to hire multi-faceted data science teams, composed of several distinct roles, each with key contributions to achieve the team’s ultimate goal."""},
        {"body": """The current confusion around what data science is can’t last. Data science teams are expensive, and without clear professional requirements and expectations, organizations will struggle to get value out of their DS teams. As of today, organizations without a well defined data science team may struggle in the following ways: """},
        {"list": ["""1. In hiring and career development, overloaded job requirements create the perceived need for an infinite string of education and certificates. Once candidates are hired, unclear expectations lead to difficulty evaluating employee performance. Articulating the description and expectations of a data science role and credentials is essential for finding quality candidates and supporting their career development. """,
                  """2. For senior leadership, job openings are often built with the wrong terminology and responsibilities, resulting in founding team members without requisite skills for scaling up the data science team. This can be compounded with inappropriate team structure and organizational goals resulting in inadequate inter-team ownership and project timeline assessments. The lack of apparent progress will frustrate the organization and lead it to conclude that data science teams aren’t capable of their expected transformational impact.""",
                  """3. In managing, unclear team workflow management leads to scope creep and inferior products. Put plainly, if you have a team of software engineers, there are studied and battle-tested approaches to organizational management, and adopting these approaches leads to success. Analyst teams have different requirements, workflows and career paths which have likewise existed for decades. Hybridizing the approaches, by viewing data scientists as not-quite engineers or not-quite analysts, creates workflow complexity and frustration among individual employees."""
                                ]},
        {'header': """Data science teams can be built from traditional roles."""},
        {'body': """Conceptualizing data science as an outcome gives a team the freedom to categorize data science jobs by their underlying functions. This is a different paradigm--where data science teams are composed of pre-existing roles that work together to achieve an organizational outcome. Some of these roles are detailed below:"""},
        {'table': {'header':['Role', 'Description', 'Category'],
                  'data': [["Data / Business Intelligence Engineer", "Transform organizational data into a useful form.", "Software Engineering"],
                           [
                            "Machine Learning Engineer",
                            "Build products (services, UIs, etc.) with ML/AI.",
                            "Software Engineering"
                           ],
                           [
                            "Database Administrator",
                            "Configure and manage databases or warehouses for ML or analysis.",
                            "Software Engineering"
                           ],
                           [   
                            "Platform Engineer",
                            "Develop internal tooling for scalably deploying and managing products.",
                            "Software Engineering"
                           ],
                           [
                            "Analyst",
                            "Produce strategic recommendations from data.",
                            "Analyst"
                           ],
                           [
                            "Statistician",
                            "Produce recommendations based on rigorous statistical analysis.",
                            "Analyst",
                           ],
                            [                       
                            "Product Manager",
                            "Manage the development of data products.",
                            "Product"
                            ]
                          
                          ]
                  }
        },
        {'body': """The standard categorizations in the rightmost column are meant to highlight the idea that data science teams can be composed of jobs which have existed for a long time. The title “data scientist” is conspicuously absent from the above categorization, since my contention is that “data scientist” isn’t a job in and of itself. """},
        {'body': """Some organizations consider a “data scientist” someone who works exclusively on a machine learning workflow: exploratory data analysis, feature engineering, and iterative model building. In cases where a team member is focusing most of their time on this workflow, the job should still be categorized as a software engineer or analyst, because a model that hasn’t been deployed or has not been analyzed does not provide any organizational value. They are providing an incomplete portion of the work, which will need to be finished by an analyst or software engineer. After model creation, an analysis needs to be performed and shared to examine driving factors (making them an analyst) or it must be deployed in an accessible service and combined with business logic (software (ML) engineer). The construction of a machine learning model is always an intermediate outcome to producing value."""},
        {'body': """Entangling a separate software team to manage model deployment or a separate analyst team to describe strategic findings creates blockers at the hand-off points between domains. If organizations want business outcomes from data science teams, then cutting off responsibility at model creation is a shortsighted approach. The organizational goal should be to hire a mixture of software engineers, analysts, and product roles to inhabit a data science team, owning value creation from start to finish. """},
        {'header': """“Data science” titles remain a necessary evil.""" },
        {'body': """While the role may not really exist as a separate job function, the prestige that the “data scientist” title holds is undeniable. This ontological investigation can’t ignore the practical limitations of role marketing and SEO. Job postings (and resumes) are written to emphasize the roles that will get the most market traction. Right now, “data scientist” is a high-traction role, for both job listers and applicants. Even though the role itself might not exist, most interviewees will label themselves as such, perpetuating the role confusion. """},
        {'body': """As organizational management learns more about data science teams, the title will become less important, and applicants will be more willing to advertise their real past roles. Data science team leaders can help by creating job titles and descriptions which include the necessary buzzwords, but limit the scope of responsibilities that the new hire is expected to support. Even without changing the job titles, keeping a good internal understanding of what the organization’s “data science” roles really are helps streamline hiring and expectation setting."""},
        {'header': """Leading companies are already adopting this paradigm."""},
        {'body': """Recently, some companies have broken the data science function into more distinct job roles, though this is usually at mature data science organizations. These role descriptions can be found in a handful of popular articles ({6}) and organizational publications ({7},{8},{9},{10},{11},{12}) that brand their data scientist roles differently, and this is a signaling mechanism that their roles are aligning with the more traditional job functions listed above."""},
        {'body': """One of the most common terms is “full-stack” data scientist ({7},{8},{9},{10},{11}) where organizations indicate that their data scientists are expected to be owners across the full spectrum of their model impacts, from ETL to model development to deployment and productization. ({12}) The advice given by these teams is often that software engineering best practices have been effective internally and learning some developer ops or working with a platform ops team is a way to “level-up” your data science. This is a clever marketing move to attract data science applicants while advertising a role that is, for all intents and purposes, a software engineer. Granted, the role requires special knowledge of machine learning, but in that regard many software engineering openings require special domain knowledge (frontend vs backend engineer, for example). I use this same terminology when talking about my data science team externally, but I lament my contribution towards job role confusion."""},
        {'header': """Smaller organizations will have less data science role separation."""},
        {'body': """At smaller organizations, data science team members might have to wear many hats: machine learning engineer, data engineer, and analyst all rolled into one. However, this doesn’t indicate the existence of a new job role, rather it underlines that when spread thin, teams need to cover the highest priority tasks regardless of job title. Just as a start-up founder might also be head of sales or marketing, a data science team member may need to function as an engineer and an analyst. My experiences with small teams have led me to believe that when a data science team member is wearing a particular hat, the best practices for that role (be it software engineering or analysis) will help them to be most effective. New or innovative workflows may not be necessary when data science jobs are split into their traditional categories above."""},
        {'header': """Create teams with diverse role coverage to get value from your data."""},
        {'body': """So, when first building out a team, or refactoring the team, think about the organizational outcomes you’re interested in for your data science team. Hiring “data scientists” is too broad of a category and without an outcome-based plan, your team is unlikely to have the right roles. Do you need a data product to be made? Hire machine learning engineers, and manage them with software engineering best practices. Need strategic insights from your data? Hire a team of analysts and manage them using analysis management best practices. Hybridizing everyone into one role will artificially increase the overhead and people necessary to execute a company’s vision for monetizing their data."""}
    ],
"references": [
    "https://hbr.org/2012/10/data-scientist-the-sexiest-job-of-the-21st-century",
    "https://datascience.berkeley.edu/about/what-is-data-science/",
    "http://www.sas.rochester.edu/dsc/about/data-science.html",
    "https://www.udacity.com/course/intro-to-data-science--ud359",
    "http://drewconway.com/zia/2013/3/26/the-data-science-venn-diagram",
    "https://hbr.org/2018/11/the-kinds-of-data-scientist#:~:text=Modeling%20scientist%3A%20Models%2C%20training%20data,the%20code%20developed%20and%20shipped",
    "https://multithreaded.stitchfix.com/blog/2019/03/11/FullStackDS-Generalists/",
    "https://syslog.ravelin.com/data-science-best-practices-843c9693db8",
    "https://ckmanalytix.com/the-full-stack-data-scientist/",
    "https://blog.dominodatalab.com/data-science-at-the-new-york-times/",
    "https://medium.com/@akiratakezawa/the-golden-path-to-become-a-full-stack-data-scientist-who-is-needed-by-in-real-industry-e865e47e3788",
    "https://multithreaded.stitchfix.com/blog/2016/03/16/engineers-shouldnt-write-etl/"
],
"tags": ['data science', 'machine learning', 'leadership', 'teams', 'management', 'art snob', 'growth', 'hiring'],
"author": {"name": "Mike Skarlinski, PhD", "title": "Art Snob Founder", "bio": "Mike has been building data products and leading data science teams over the last 9 years."},
"publish_date":'Feb. 22, 2021'
    }

In [7]:
clean_title = article['title'].lower().replace('%2c', ',').replace('\'', '').replace(' ','_')
dsi.update(data_list=[{'article': json.dumps(article)}], ids=[clean_title], kind='artsnob-blog-articles', exclude_from_indexes=['article'])